In [1]:
import _pickle as cPickle
import os
import numpy as np
import nltk
import nltk.corpus
import pandas as pd

# Truecasing

A small fraction of the dpa articles were missing all uppercases. To fix this, we used a model for truecasing by Nils Reimers
(https://github.com/nreimers) which is based on Lucian Vlad Lita et al. (2003).

### NOTE: The following model was taken from https://github.com/nreimers/truecaser. 

In [2]:
# Select path to model
path = 'D:\\Studium\\PhD\\Media Tenor\\Preprocessing\\Truecasing\\truecaser-master'
os.chdir(path)

In [3]:
from TrainFunctions import *
from EvaluateTruecaser import defaultTruecaserEvaluation

In [4]:
# Initialize frequency distributions for bigrams
uniDist = nltk.FreqDist()
backwardBiDist = nltk.FreqDist() 
forwardBiDist = nltk.FreqDist() 
trigramDist = nltk.FreqDist() 
wordCasingLookup = {}

In [5]:
# load trainings data
data = pd.read_csv(r'D:\\Studium\\PhD\\Media Tenor\\Results\\dpa_prepro_step6.csv', dtype = {'source': 'string', 'genre': 'string'})#, encoding = 'utf-8', sep=';')
data.dropna(subset = ['texts'], inplace = True)

In [6]:
# Select data for training truecasing model and tokenize text
data_sample = [nltk.word_tokenize(sentence) for sentence in data['texts'][1900000:1910000]]

In [7]:
# train truecaser model
updateDistributionsFromSentences(data_sample, wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)

# Save distributions
f = open('distributions.obj', 'wb')
cPickle.dump(uniDist, f, -1)
cPickle.dump(backwardBiDist, f, -1)
cPickle.dump(forwardBiDist, f, -1)
cPickle.dump(trigramDist, f, -1)
cPickle.dump(wordCasingLookup, f, -1)
f.close()

In [8]:
# Evaluate truecaser model
defaultTruecaserEvaluation(wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist)

['Obwohl', 'der', 'Erwerb', 'von', 'Wohnungen', 'seit', '1974', 'fuer', 'Auslaender', 'weitgehend', 'verboten', 'sei', ',', 'haetten', 'diese', 'in', 'zahlreichen', 'Faellen', 'ueber', 'oesterreichische', 'Strohmaenner', 'Wohnungen', 'gekauft', '.']
['Obwohl', 'der', 'Erwerb', 'von', 'Wohnungen', 'seit', '1974', 'Fuer', 'Auslaender', 'weitgehend', 'verboten', 'sei', ',', 'Haetten', 'diese', 'in', 'zahlreichen', 'Faellen', 'Ueber', 'Oesterreichische', 'Strohmaenner', 'Wohnungen', 'gekauft', '.']
-------------------
['So', 'seien', 'Menschen', 'im', 'oesterreichischen', 'Altersheim', 'als', 'Kaeufer', 'aufgetreten', ',', 'die', 'ihr', 'Eigentum', 'dem', 'deutschen', 'Anleger', 'vererbt', 'haetten', '.']
['So', 'seien', 'Menschen', 'im', 'oesterreichischen', 'Altersheim', 'als', 'Kaeufer', 'aufgetreten', ',', 'die', 'ihr', 'Eigentum', 'dem', 'deutschen', 'Anleger', 'vererbt', 'Haetten', '.']
-------------------
['Bonn', '(', 'dpa', ')', '-', 'Die', 'Zusammensetzung', 'der', 'künftigen', '

In [9]:
from Truecaser import *
import nltk
import string
import PredictTruecaser
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [10]:
# Define a test sentence for demonstration
test_sentence = 'dieses wort ist falsch geschrieben. dieser satz ist ein test.'

In [11]:
corr_sent = getTrueCase(nltk.word_tokenize(test_sentence), 'title', wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, 
                       trigramDist)

corr_sent = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in corr_sent]).strip()
#corr_sent = TreebankWordDetokenizer().detokenize(corr_sent)
print(corr_sent)

Dieses Wort ist falsch geschrieben. Dieser Satz ist ein Test.


In [12]:
# Load articles which we want to correct
data_truecase = pd.read_csv(r'D:\\Studium\\PhD\\Media Tenor\\Results\\dpa_case_fix.csv', dtype = {'source': 'string', 'genre': 'string'}, encoding = 'utf-8', sep=';')
data_truecase.dropna(subset = ['texts'], inplace = True)

In [13]:
data_truecase['texts'].iloc[15]

"new york (vwd) - die aktienboerse in new york konnte am donnerstag ihre im sitzungsverlauf gesehenen kursgewinne nahezu behaupten. der dow-jones-index fuer 30 industriewerte schloss mit 2.643,07 um 24,01 punkte bzw 0,92 prozent ueber vortagesschluss. auch die breiter gefassten marktbarometer sowie die branchenindizes lagen im plusterritorium. umgesetzt wurden 223,15 (169,44) millionen aktien, wobei die kursgewinner gegenueber den -verlierern eine fuehrung von rund drei zu eins behaupteten. haendler fuehrten die aufschlaege auf hoffnungen am markt zurueck, wonach das federal reserve board in kuerze weitere zinssenkungen vornehmen werde, um die us-konjunktur anzukurbeln. die hoffnungen waren am mittwoch durch aeusserungen von fed-chairman alan greenspan genaehrt worden, der angesichts des 'credit crunch' der banken in den usa eine senkung der us-tagesgeldsaetze befuerwortet hatte. ausserdem, so die boersianer, habe der markt eine stuetze durch die wieder nachgebenden oelpreise erhalten.

In [14]:
# Use trained truecaser model to correct articles
truecase_art = []

for text in data_truecase['texts']:
    truecase_art.append(getTrueCase(nltk.word_tokenize(text), 'title', wordCasingLookup, uniDist, backwardBiDist, forwardBiDist, trigramDist))

In [15]:
truecase_art = ["".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in article]).strip() for article in truecase_art]

In [16]:
data_truecase['texts'] = truecase_art

In [17]:
data_truecase['texts'].iloc[15]

"New York( Vwd)- Die Aktienboerse in New York konnte am Donnerstag ihre im Sitzungsverlauf Gesehenen Kursgewinne nahezu behaupten. Der Dow-Jones-Index Fuer 30 Industriewerte schloss mit 2.643,07 um 24,01 Punkte bzw 0,92 Prozent Ueber Vortagesschluss. Auch die breiter gefassten Marktbarometer sowie die Branchenindizes lagen im Plusterritorium. umgesetzt wurden 223,15( 169,44) Millionen Aktien, wobei die Kursgewinner Gegenueber den -Verlierern eine Fuehrung von rund drei zu eins behaupteten. Haendler Fuehrten die Aufschlaege auf Hoffnungen am Markt Zurueck, wonach das Federal Reserve Board in Kuerze weitere Zinssenkungen vornehmen werde, um die US-Konjunktur anzukurbeln. Die Hoffnungen waren am Mittwoch durch Aeusserungen von Fed-Chairman Alan Greenspan Genaehrt worden, der angesichts des'Credit Crunch' der Banken in den USA eine Senkung der Us-Tagesgeldsaetze Befuerwortet hatte. Ausserdem, so die Boersianer, habe der Markt eine Stuetze durch die wieder nachgebenden Oelpreise erhalten. p

In [18]:
data_truecase.to_csv('D:\\Studium\\PhD\\Media Tenor\\Results\\dpa_cases_fixed.csv', encoding='utf-8-sig', sep = ';')